In [1]:
pip install streamlit streamlit_folium openai python-dotenv geopandas


  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   --- ------------------------------------ 1.8/19.2 MB 14.4 MB/s eta 0:00:02
   --------- ------------------------------ 4.5/19.2 MB 12.2 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/19.2 MB 9.3 MB/s eta 0:00:02
   -------------- ------------------------- 7.1/19.2 MB 8.9 MB/s eta 0:00:02
   -------------------- ------------------- 9.7/19.2 MB 9.4 MB/s eta 0:00:02
   ------------------------ --------------- 11.8/19.2 MB 9.6 MB/s eta 0:00:01
   ----------------------------- ---------- 14.2/19.2 MB 9.8 MB/s eta 0:00:01
   -------------------------------- ------- 15.5/19.2 MB 9.5 MB/s eta 0:00:01
   --------------------------------- ------ 16.3/19.2 MB 8.7 MB/s eta 0:00:01
   ------------------------------------- -- 17.8/19.2 MB 8.5 MB/s eta 0:00:01
   -----------------------------

In [ ]:
import json
import sys

In [ ]:
from chatbot_logic import (
    load_chunks_from_jsonl,
    normalize_text,
    extract_query_parameters,
    find_relevant_chunks
    # send_question_to_openai # Opcional, si quieres probarlo aquí
)

In [ ]:
# Cargar los chunks una vez
all_docs_chunks = load_chunks_from_jsonl('salida_chunks_final.jsonl') # Asegúrate que la ruta sea correcta
print(f"Cargados {len(all_docs_chunks)} chunks.")
if all_docs_chunks:
    print("Ejemplo del primer chunk:")
    print(json.dumps(all_docs_chunks[0], indent=2, ensure_ascii=False))
else:
    print("¡No se cargaron chunks! Verifica la ruta y el archivo.")


In [ ]:
test_text = "Región Áncash con Énfasis"
normalized = normalize_text(test_text)
print(f"Original: '{test_text}' -> Normalizado: '{normalized}'")
# Esperado: region ancash con enfasis


In [ ]:
print("\n--- Probando extract_query_parameters ---")
queries = [
    "informe de corrupcion en lima 2019",
    "tienes datos sobre la region cuzco",
    "junin en 2020",
    "reportes de ayacucho y apurimac entre 2018 y 2020 sobre obras publicas",
    "que paso en la municipalidad de huamanga" # Ejemplo sin año/región clara
]

for q in queries:
    params = extract_query_parameters(q)
    print(f"Query: '{q}'\n  Params: {params}")


In [ ]:
# Celda 4: Probar find_relevant_chunks (con los prints de depuración activados dentro de la función)
print("\n--- Probando find_relevant_chunks ---")

# Pregunta 1
question1 = "informe de corrupcion en lima 2019"
print(f"\nBuscando para: '{question1}'")
result1 = find_relevant_chunks(question1, all_docs_chunks, max_chunks=5)
if result1["needs_more_specificity"]:
    print("Resultado: Necesita más especificidad.")
elif not result1["chunks"]:
    print(f"Resultado: No se encontraron chunks. Razón: {result1.get('no_data_for_filter') or result1.get('no_data_for_filter_after_score', 'desconocida')}")
    if result1.get("params"): print(f"  Parámetros usados: {result1.get('params')}")
else:
    print(f"Resultado: Encontrados {len(result1['chunks'])} chunks.")
    # for i, chunk in enumerate(result1["chunks"]):
    #     print(f"  Chunk {i+1} (Informe: {chunk.get('metadata', {}).get('numero_informe', 'N/A')}, Región: {chunk.get('metadata', {}).get('region', 'N/A')}, Año: {chunk.get('metadata', {}).get('year', 'N/A')})")
    #     print(f"    Texto: {chunk.get('chunk_text', '')[:150]}...") # Imprime un fragmento

# Pregunta 2
question2 = "tienes datos sobre la region cuzco"
print(f"\nBuscando para: '{question2}'")
result2 = find_relevant_chunks(question2, all_docs_chunks, max_chunks=5)
# ... (mismo manejo de resultados que para question1) ...
if result2["needs_more_specificity"]:
    print("Resultado: Necesita más especificidad.")
elif not result2["chunks"]:
    print(f"Resultado: No se encontraron chunks. Razón: {result2.get('no_data_for_filter') or result2.get('no_data_for_filter_after_score', 'desconocida')}")
    if result2.get("params"): print(f"  Parámetros usados: {result2.get('params')}")
else:
    print(f"Resultado: Encontrados {len(result2['chunks'])} chunks.")


# Pregunta 3 (para Junín)
question3 = "informes para junin en 2020"
print(f"\nBuscando para: '{question3}'")
result3 = find_relevant_chunks(question3, all_docs_chunks, max_chunks=5)
# ... (mismo manejo de resultados) ...
if result3["needs_more_specificity"]:
    print("Resultado: Necesita más especificidad.")
elif not result3["chunks"]:
    print(f"Resultado: No se encontraron chunks. Razón: {result3.get('no_data_for_filter') or result3.get('no_data_for_filter_after_score', 'desconocida')}")
    if result3.get("params"): print(f"  Parámetros usados: {result3.get('params')}")
else:
    print(f"Resultado: Encontrados {len(result3['chunks'])} chunks.")

